In [16]:
"""
@authors: faurand, chardes, ehagensieker
"""

import tensorflow_datasets as tfds
import tensorflow_text as tf_text
import tensorflow as tf
import sentencepiece as sp
import tqdm.notebook as notebook
import os
import io 
import re
import datetime
import numpy as np

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [17]:
shakespeare_url='https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
filepath=tf.keras.utils.get_file('shakespeare.txt',shakespeare_url)

with open(filepath) as f:
    txt=f.read()

txt = txt[:500000]
print(txt[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [18]:
txt = txt.replace("\n", " ").lower()
txt = re.sub('[^A-Za-z]+', ' ', txt) 

print(f"converted: {txt[:500]} \n")

converted: first citizen before we proceed any further hear me speak all speak speak first citizen you are all resolved rather to die than to famish all resolved resolved first citizen first you know caius marcius is chief enemy to the people all we know t we know t first citizen let us kill him and we ll have corn at our own price is t a verdict all no more talking on t let it be done away away second citizen one word good citizens first citizen we are accounted poor citizens the patricians good what auth 



In [19]:
VOCAB_SIZE = 4000
M = 32
KEY_DIM = 128

sp.SentencePieceTrainer.train(input = filepath, model_prefix = 'tokenizer_model', model_type='unigram', vocab_size = VOCAB_SIZE)

# deserialize the trained model file to load it in the correct format
trained_tokenizer_model = tf.io.gfile.GFile('tokenizer_model.model', "rb").read()

# load the model as a tokenizer that can be used inside a tensorflow model
tokenizer = tf_text.SentencepieceTokenizer(
    model=trained_tokenizer_model, out_type=tf.int32, nbest_size=-1, alpha=1, reverse=False,
    add_bos=False, add_eos=False, return_nbest=False, name=None
)

tokenized = tokenizer.tokenize(txt)

In [20]:
txt = tf_text.sliding_window(tokenized, M+1)

In [21]:
train_data = tf.data.Dataset.from_tensor_slices((txt, txt))

train_data = train_data.map(lambda x, y: (x[:M], y[1:]))
train_data = train_data.shuffle(1028).batch(32, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

train_data

<PrefetchDataset element_spec=(TensorSpec(shape=(32, 32), dtype=tf.int32, name=None), TensorSpec(shape=(32, 32), dtype=tf.int32, name=None))>

In [22]:
class Embedding(tf.keras.layers.Layer): 
    def __init__(self, vocab_size, seq_len, embedding_size = 64): 
        super(Embedding, self).__init__()

        self.embed_token = tf.keras.layers.Embedding(vocab_size, embedding_size, input_length = 1)
        self.embed_pos = tf.keras.layers.Embedding(seq_len, embedding_size, input_length = 1)

    def call(self, inputs): 
        x = self.embed_token(inputs)
        length = tf.shape(inputs)[1]
        tensor = tf.range(0, length)
        pos = self.embed_pos(tensor)

        return pos + x

In [23]:
class Transformer(tf.keras.layers.Layer): 
    def __init__(self, num_heads, embedding_size = 64): 
        super(Transformer, self).__init__()

        self.multiHead = tf.keras.layers.MultiHeadAttention(num_heads = num_heads, key_dim = embedding_size)

        self.dense1 = tf.keras.layers.Dense(units = embedding_size, activation = 'relu')
        self.dense2 = tf.keras.layers.Dense(units = embedding_size)

        self.dropout1 = tf.keras.layers.Dropout(rate = 0.1)
        self.dropout2 = tf.keras.layers.Dropout(rate = 0.1)

        self.norm1 = tf.keras.layers.LayerNormalization(epsilon = 1e-6)
        self.norm2 = tf.keras.layers.LayerNormalization(epsilon = 1e-6)

    def call(self, inputs, training = None): 

        x = self.multiHead(inputs, inputs, use_causal_mask = True)
        x = self.dropout1(x, training = training)
        x = x + inputs
        ln_out = self.norm1(x, training = training)
        x = self.dense1(ln_out)
        x = self.dense2(x)
        x = self.dropout2(x, training = training)
        x = ln_out + x

        return self.norm2(x, training = training)

In [24]:
class MyModel(tf.keras.Model): 
    def __init__(self, tokenizer, num_heads, vocab_size, seq_len, embedding_size = 64): 
        super(MyModel, self).__init__()

        self.tokenizer = tokenizer
        self.num_heads = num_heads
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.seq_len = seq_len
        
        self.optimizer = tf.keras.optimizers.Adam(0.001)
        self.loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

        self.metrics_list = [tf.keras.metrics.Mean(name="loss"),
                             tf.keras.metrics.CategoricalAccuracy(name="acc")]

        self.embedding_layer = Embedding(self.vocab_size, self.seq_len)
        self.transformer_block = Transformer(self.num_heads)
        self.dense = tf.keras.layers.Dense(units = self.vocab_size)

    def call(self, inputs, training = None):
        y = self.embedding_layer(inputs)
        y = self.transformer_block(y, training=training)
        y = self.dense(y)

        return y

    def reset_metrics(self): 
        for metric in self.metrics_list: 
            metric.reset_states()

    @tf.function
    def train_step(self, inputs):
        
        x, targets = inputs
        
        with tf.GradientTape() as tape:
            predictions = self(x, training = True)
            loss = self.loss_func(targets, predictions) + tf.reduce_sum(self.losses)
      
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        # update loss metric
        self.metrics_list[0].update_state(loss)

        # for all metrics except loss, update states (accuracy etc.)
        for metric in self.metrics_list[1:]:
            metric.update_state(targets, predictions)

        # Return a dictionary mapping metric names to current value
        return {m.name: m.result() for m in self.metrics_list}

    def gen_text(self, prompt, output_length, top_k = 50):

        prompt = self.tokenizer.tokenize(prompt)

        while len(prompt) <= output_length:
            
            tokens = tf.expand_dims(prompt, axis = 0)
            tokens = tf.pad(tokens, tf.constant([[0, 0, ], [self.seq_len - len(prompt), 0]]), 'CONSTANT')
            logit = self.call(tokens)

            logits, indices = tf.math.top_k(logit, k=100, sorted=True)
            indices = tf.squeeze(indices)
            logits = tf.squeeze(logits)
            
            next_sample = tf.random.categorical(logits, 1)
            next_sample = tf.squeeze(next_sample)[-1].numpy()

            new_token = indices[-1, next_sample]
            new_token = tf.expand_dims(new_token, axis = -1)
            prompt = tf.concat([prompt, new_token], axis = -1)

        out = self.tokenizer.detokenize(prompt)
        print(out)
        
        return out

In [25]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

config_name = f'RUN'
train_log_path = f"logs/tests/{config_name}/{current_time}/train"

# log writer
train_summary_writer = tf.summary.create_file_writer(train_log_path)

model = MyModel(tokenizer, 4, VOCAB_SIZE, M)

In [26]:
for epoch in range(100): 
    print(f'Epoch {epoch}: ')

    for inputs in notebook.tqdm(train_data, position = 0, leave = True):
        metrics = model.train_step(inputs)
  
    gen_text = model.gen_text("Before we proceed", output_length = 24, top_k = 100)

    with train_summary_writer.as_default(): 
        for metric in model.metrics_list:
            tf.summary.scalar(f"{metric.name}", metric.result(), step=epoch)
            tf.summary.text(name = "generated text", data = gen_text, step = epoch)

    print([f"train_{key}: {value.numpy()}" for (key, value) in metrics.items()])
    model.reset_metrics()
    print("\n")

Epoch 0: 


  0%|          | 0/3895 [00:00<?, ?it/s]

tf.Tensor(b'Before we proceed prove adintious of it of him for a letter romeo say nay will stand in', shape=(), dtype=string)
['train_loss: 3.773573160171509', 'train_acc: 0.01279936172068119']


Epoch 1: 


  0%|          | 0/3895 [00:00<?, ?it/s]

tf.Tensor(b'Before we proceeds and send dainetverss and badona romeo what thou', shape=(), dtype=string)
['train_loss: 3.4568140506744385', 'train_acc: 0.011980503797531128']


Epoch 2: 


...

tf.Tensor(b'Before we proceedion nurse dare she yes that ye should saw a man here this day of a little first messengering bones', shape=(), dtype=string)
['train_loss: 2.2408852577209473', 'train_acc: 0.010559662245213985']


Epoch 98: 


  0%|          | 0/3895 [00:00<?, ?it/s]

tf.Tensor(b'Before we proceed s watery beams her whip and telld and then for the love doth likeness like nurse there to', shape=(), dtype=string)
['train_loss: 2.2443926334381104', 'train_acc: 0.010548379272222519']


Epoch 99: 


  0%|          | 0/3895 [00:00<?, ?it/s]

tf.Tensor(b'Before we proceedion served in his suit i am a sure we will tell tell thee with night and play it for it', shape=(), dtype=string)
['train_loss: 2.2419211864471436', 'train_acc: 0.010562920942902565']


